In [9]:
from utils import extract_all_features_for_count_threshold, extract_features_for_dist_threshold, get_custom_matrix_features
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
# read in data
from utils import get_matrix_features
path_to_data = os.path.abspath('data')
path_to_colonies = os.path.abspath('data/colonies')

In [10]:

import pandas as pd
feature_list = ['']
threshold_mode = 'count'
num_nn = 4
# colony 0
colony0_segmentation_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/data/V2022_09_19_HTB2_mCh_MYO1-GFP_50_ms/FOV0_segmentation_T0_to_T146_trimmed.h5'
colony0_lineage_GT_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/data/V2022_09_19_HTB2_mCh_MYO1-GFP_50_ms/FOV0_lineage_T0_to_T146.csv'
colony0_lineage_gt = pd.read_csv(colony0_lineage_GT_path).rename(
    columns={'parent_id': 'parent_GT'})
colony0_lineage_gt['colony'] = [0 for i in range(len(colony0_lineage_gt))]

# colony0_features, f_list = extract_all_features_for_count_threshold(
#     colony0_segmentation_path, num_nn=num_nn)
# colony0_features['colony'] = [0 for i in range(len(colony0_features))]

# colony0_matrix_features = get_custom_matrix_features(
#     colony0_features, colony0_lineage_gt, f_list)

# colony 1 to 5
# load the 5 colony ground truth
colonies_gt = pd.DataFrame()
for i in [1, 2, 3, 4, 5]:
    temp_colony = pd.read_csv(os.path.join(
        path_to_colonies, 'colony00{}_lineage.csv'.format(i)))
    temp_colony['colony'] = i
    colonies_gt = pd.concat([colonies_gt, temp_colony])
colonies_gt = colonies_gt.reset_index(drop=True)
colonies_gt.rename(columns={'# parent_id': 'parent_GT'}, inplace=True)


# extract features for each colony and save them to a csv file. This takes a while. But after this, we only need to load the csv files. every time.
    # save to csv
    # colony_candidate_features[i-1].to_csv(os.path.join(
    #     path_to_colonies, 'colony00{}_candidate_features_19_{}_{}_nn_{}_threshold.csv'.format(i, threshold_mode, num_nn, 10)), index=False)
    # colony_candidate_features[i-1] = pd.read_csv(os.path.join(path_to_colonies, 'colony00{}_candidate_features_{}_{}_nn.csv'.format(i, threshold_mode, num_nn)))



In [11]:

_19_f_list = ['dist_0','dist_max','dist_min','dist_std','poly_fit_budcm_candidcm','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']
# merge colonies features and colony0 features and save in one csv file
all_candidate_features = pd.read_csv(os.path.join(path_to_data, 'all_candidate_features_19_{}_{}_nn_{}_thresh.csv'.format(threshold_mode, num_nn, 10)))
all_gt = pd.concat([colony0_lineage_gt,colonies_gt]).reset_index()
# all_candidate_features.to_csv(os.path.join(path_to_data, 'all_candidate_features_19_{}_{}_nn_{}_thresh.csv'.format(threshold_mode, num_nn, 10)), index=False)
all_matrix_features = get_custom_matrix_features(all_candidate_features, all_gt, _19_f_list)
all_matrix_features

,index,parent_GT,bud_id,time_index,colony,features,candidates,parent_index_in_candidates
4,4,1,5,18,0,"[[2.23606797749979, 2.8284271247461903, 2.0, 0...","[1, 3, 2, 4]",0
5,5,3,6,18,0,"[[1.0, 3.0, 1.0, 0.7071067811865476, 3.1110787...","[3, 4, 2, 1]",0
6,6,2,7,20,0,"[[1.0, 3.0, 1.0, 0.7512255508302275, 0.9423480...","[2, 3, 4, 6]",0
7,7,4,8,22,0,"[[1.0, 2.0, 1.0, 0.5, 2.107542585550348, 2.122...","[4, 7, 6, 2]",0
8,8,1,9,32,0,"[[2.0, 2.0, 1.0, 0.4330127018922193, 1.7589301...","[1, 5, 3, 2]",0
...,...,...,...,...,...,...,...,...
867,499,22,34,164,5,"[[2.0, 3.0, 2.0, 0.4330127018922193, 1.1800917...","[22, 25, 8, 11]",0
868,500,13,35,166,5,"[[2.0, 3.0, 2.0, 0.4611141570664294, 0.8067932...","[13, 16, 10, 4]",0
869,501,18,36,169,5,"[[2.23606797749979, 4.123105625617661, 2.23606...","[18, 9, 2, 32]",0
870,502,3,37,175,5,"[[2.0, 2.23606797749979, 2.0, 0.10222043276741...","[18, 26, 3, 27]",2


# train


In [12]:
from nn_model import BudDataset, LineageNN, train_nn, test_nn, cv_nn


In [13]:
WANDB_NOTEBOOK_NAME = 'test_features'

In [14]:

config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [19*4, 128, 5], 'augment': True , 'features':f_list,'train_set': 'all'}
models, accuracies = cv_nn(all_matrix_features, config=config)

config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [76, 128, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_first', 'position_bud_std', 'orientation_bud_first', 'orientation_bud_last', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 1}
yes. I am reloaded


best_accuracy,▁▄▅▆▇███████████████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
eval_accuracy,▁▄▅▆▇██▆████▆▆▇▇▇▇▅▇
lr,██▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▁▁
patience,▁▁▁▁▁▁▂▂▁▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▆▆▆▇▇▇▇▇▇██▇▇█████
best_accuracy,0.8303
epoch,19
eval_accuracy,0.81212
lr,0.008
patience,10


early stopping at  25 LR:  0.0074026000000000005
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [76, 128, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_first', 'position_bud_std', 'orientation_bud_first', 'orientation_bud_last', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 2}
yes. I am reloaded


best_accuracy,▁▁▃▅▅▆▆▆▆▆▇▇▇▇███████████
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
eval_accuracy,▂▁▃▆▆▆▄▅▄▄▇▇▆▇██▆▆▆▇▇█▅██
lr,██▇▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁
patience,▁▂▁▁▂▁▂▂▃▄▁▂▂▃▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇█▇█████
best_accuracy,0.85542
epoch,24
eval_accuracy,0.8494
lr,0.0075
patience,10


early stopping at  34 LR:  0.006503500000000001
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [76, 128, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_first', 'position_bud_std', 'orientation_bud_first', 'orientation_bud_last', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 3}
yes. I am reloaded


...

early stopping at  39 LR:  0.006004000000000001
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [76, 128, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_first', 'position_bud_std', 'orientation_bud_first', 'orientation_bud_last', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 4}
yes. I am reloaded


best_accuracy,▁▃▃▅▆▆▆▆▇▇▇▇▇▇▇▇▇██████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval_accuracy,▁▃▃▅▆▅▅▆▇▇▆▆▆▆▇▆▆█▇▇▇▇▆█▇█▇▇█▇█▇▇▆███▇█
lr,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
patience,▁▁▁▁▁▂▂▃▁▁▂▂▃▄▅▅▆▁▂▂▃▄▅▅▆▇▇█▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████
best_accuracy,0.87273
epoch,38
eval_accuracy,0.86061
lr,0.0061
patience,10


early stopping at  16 LR:  0.0083017
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [76, 128, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_first', 'position_bud_std', 'orientation_bud_first', 'orientation_bud_last', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 5}
yes. I am reloaded


best_accuracy,▁▃▅▅▆███████████
epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
eval_accuracy,▁▃▅▁▆█▃▃▅▃▅▁▃▅▆▆
lr,██▇▇▆▆▅▅▄▄▃▃▂▂▁▁
patience,▁▁▁▂▁▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▆▆▆▇▇▇▇▇▇████
best_accuracy,0.83636
epoch,15
eval_accuracy,0.81212
lr,0.0084
patience,10


early stopping at  34 LR:  0.006503500000000001
accuracy:  0.8522964585615188 +/- 0.016560518164735638


In [15]:
# baseline (0.8486527929901424 +/- 0.01593595415915115)
f_list = ['dist_0', 'dist_max','poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_first','position_bud_std','orientation_bud_first','orientation_bud_last', 'plyfit_orientation_bud']
matrix_feat_custom = get_custom_matrix_features(all_candidate_features, all_gt, f_list)
config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all'}
models, accuracies = cv_nn(matrix_feat_custom, config=config, seed=0)

config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_first', 'position_bud_std', 'orientation_bud_first', 'orientation_bud_last', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 1}
yes. I am reloaded


best_accuracy,▁▄▄▅▅▅▅▅▆▇▇▇▇▇▇▇▇▇▇▇██████████████
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇███
eval_accuracy,▁▄▃▅▃▅▂▅▆▇▇▇▆▅▇▆▇▇▄▆█▇▅█▆▇██▆█▆▆▆▄
lr,███▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁
patience,▁▁▂▁▂▂▃▄▁▁▁▂▂▃▄▅▅▆▇▇▁▂▂▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▅▆▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇██████
best_accuracy,0.8303
epoch,33
eval_accuracy,0.78182
lr,0.0066
patience,10


early stopping at  17 LR:  0.0082018
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_first', 'position_bud_std', 'orientation_bud_first', 'orientation_bud_last', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 2}
yes. I am reloaded


best_accuracy,▁▁▆▆▆▆███████████
epoch,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
eval_accuracy,▃▁▇▇▆▃█▆▆▆▆█▆▇▇▆▅
lr,██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
patience,▁▂▁▁▂▂▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▅▆▆▇▇▇▇▇█▇█████
best_accuracy,0.83735
epoch,16
eval_accuracy,0.78916
lr,0.0083
patience,10


early stopping at  28 LR:  0.0071029000000000005
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_first', 'position_bud_std', 'orientation_bud_first', 'orientation_bud_last', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 3}
yes. I am reloaded


...

early stopping at  40 LR:  0.005904100000000001
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_first', 'position_bud_std', 'orientation_bud_first', 'orientation_bud_last', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 4}
yes. I am reloaded


best_accuracy,▁▂▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇███████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval_accuracy,▁▂▅▄▄▆▆▆▅▅▆▆▅▇▆▇▆▇▆▇▇▆▇▇▇█▇█▇██▇███▇████
lr,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
patience,▁▁▁▂▂▁▁▂▂▃▄▅▅▁▂▂▃▄▅▁▂▂▃▄▅▁▂▂▃▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████
best_accuracy,0.87273
epoch,39
eval_accuracy,0.86061
lr,0.006
patience,10


early stopping at  23 LR:  0.0076024000000000005
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_first', 'position_bud_std', 'orientation_bud_first', 'orientation_bud_last', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 5}
yes. I am reloaded


best_accuracy,▁▁▁▅▅▅▅▅▆▆▆▇███████████
epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇██
eval_accuracy,▅▁▂▆▅▇▆▆▇▆▇▇█▇███▇▆▇▆▅▇
lr,██▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▂▂▂▁▁
patience,▁▂▂▁▂▁▂▂▁▂▂▁▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▅▆▆▆▇▇▇▇▇▇███████████
best_accuracy,0.8303
epoch,22
eval_accuracy,0.81818
lr,0.0077
patience,10


early stopping at  20 LR:  0.0079021
accuracy:  0.8474698795180723 +/- 0.018429748381058802


In [ ]:
# add custom filling features (0.8631982475355968 +/- 0.015594615266022033)
f_list = ['dist_0', 'dist_max','poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_first','position_bud_std','orientation_bud_first','orientation_bud_last', 'plyfit_orientation_bud']
matrix_feat_custom = get_custom_matrix_features(all_candidate_features, all_gt, f_list, filling_features=[100,100,0,0,0,-1,-1,-1,-1,-1])
config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all'}
models, accuracies = cv_nn(matrix_feat_custom, config=config)

In [ ]:
# Add std of dist to see the difference (0.8510770354143847 +/- 0.017908805065818198)
f_list = ['dist_0', 'dist_max', 'dist_std','poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_first','position_bud_std','orientation_bud_first','orientation_bud_last', 'plyfit_orientation_bud']
matrix_feat_custom = get_custom_matrix_features(all_candidate_features, all_gt, f_list)
config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all'}
models, accuracies = cv_nn(matrix_feat_custom, config=config)

In [ ]:
# Add std of dist and custom filling features (0.8546)
# add custom filling features
f_list = ['dist_0', 'dist_max', 'dist_std','poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_first','position_bud_std','orientation_bud_first','orientation_bud_last', 'plyfit_orientation_bud']
matrix_feat_custom = get_custom_matrix_features(all_candidate_features, all_gt, f_list, filling_features=[100,100,10,0,0,0,-1,-1,-1,-1,-1])
config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all',}
models, accuracies = cv_nn(matrix_feat_custom, config=config)

In [ ]:

f_list = ['dist_0', 'dist_max','poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_first','position_bud_std','orientation_bud_first','orientation_bud_last', 'plyfit_orientation_bud']
matrix_feat_custom = get_custom_matrix_features(all_candidate_features, all_gt, f_list, filling_features=[100, 100, 0,0,0,-1,10,-1,10,-1])
config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all'}
models, accuracies = cv_nn_with_lr_scheduler(matrix_feat_custom, config=config)

In [ ]:
# for the full list with custom filling features (0.8523037604965316 +/- 0.01243160815720214)
f_list = ['dist_0','dist_max','dist_min','dist_std','poly_fit_budcm_candidcm','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']

matrix_feat_custom = get_custom_matrix_features(all_candidate_features, all_gt, f_list, filling_features=[10,10,10,10,0,0,0,10,-1,-1,-1,-1,10,-1,-1,-1,-1,10,-1])
config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all',}
models, accuracies = cv_nn(matrix_feat_custom, config=config)

# remove features one by one to see how they change


In [ ]:
# no filling
full_list = ['dist_0','dist_max','dist_min','dist_std','poly_fit_budcm_candidcm','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']
results = {}
for each in full_list:
    f_list = list(set(full_list)-set([each]))
    results[each] = 0
    matrix_feat_custom = get_custom_matrix_features(all_candidate_features, all_gt, f_list)
    config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all',
    'Group': 'feature_evaluation', 'deleted_feature': each}
    models, accuracies = cv_nn(matrix_feat_custom, config=config)
    results[each] = (np.mean(accuracies))

f_list = full_list
matrix_feat_custom = get_custom_matrix_features(all_candidate_features, all_gt, f_list)
config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all',
'Group': 'feature_evaluation', 'deleted_feature': each}
models, accuracies = cv_nn(matrix_feat_custom, config=config)
results['full'] = (np.mean(accuracies))

In [ ]:

# Sort the keys based on the values in descending order
sorted_keys = sorted(results, key=results.get, reverse=True)

# Create a new dictionary with the sorted keys and values
sorted_dict = {k: results[k] for k in sorted_keys}

sorted_dict


In [ ]:
# Run removal of features af=gain to make sure that the order is correct
# no filling
full_list = ['dist_0','dist_max','dist_min','dist_std','poly_fit_budcm_candidcm','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']
results2 = {}
for each in full_list:
    f_list = list(set(full_list)-set([each]))
    results2[each] = 0
    matrix_feat_custom = get_custom_matrix_features(all_candidate_features, all_gt, f_list)
    config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all',
    'Group': 'feature_evaluation', 'deleted_feature': each}
    models, accuracies = cv_nn(matrix_feat_custom, config=config)
    results2[each] = (np.mean(accuracies))

f_list = full_list
matrix_feat_custom = get_custom_matrix_features(all_candidate_features, all_gt, f_list)
config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all',
'Group': 'feature_evaluation', 'deleted_feature': each}
models, accuracies = cv_nn(matrix_feat_custom, config=config)
results2['full'] = (np.mean(accuracies))

In [ ]:
# Sort the keys based on the values in descending order
sorted_keys = sorted(results2, key=results.get, reverse=True)

# Create a new dictionary with the sorted keys and values
sorted_dict = {k: results2[k] for k in sorted_keys}

sorted_dict


In [ ]:
# for the full list minus bad ones based on above with filling (0.8668492150419862 +/- 0.016061070413138513)
f_list = ['dist_0','dist_std','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']

matrix_feat_custom = get_custom_matrix_features(all_candidate_features, all_gt, f_list, filling_features=[10,10,10,10,0,0,0,10,-1,-1,-1,-1,10,-1,-1,-1,-1,10,-1])
config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all',}
models, accuracies = cv_nn(matrix_feat_custom, config=config)

In [ ]:
# for the full list minus bad ones based on above without filling (0.8668492150419862 +/- 0.016061070413138513)
f_list = ['dist_0','dist_std','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']

matrix_feat_custom = get_custom_matrix_features(all_candidate_features, all_gt, f_list,)
config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all',}
models, accuracies = cv_nn(matrix_feat_custom, config=config)

## test above model on external subset


In [16]:
f_list = ['dist_0','dist_std','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']
external_subset_lineage_GT_path_edited = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/lineage_budlum_edited_by_code.csv'
external_subset_features_path = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/features/features_19.csv'
external_subset_segmentation_path = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/segmentation2/stacked/mask_stack _edited.h5'
threshold_mode = 'count'
num_nn = 4
path_to_features = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/'

external_subset_features ,_ =  extract_all_features_for_count_threshold(
    external_subset_segmentation_path, num_nn=num_nn)


[16:55:01] WARNING: /croot/xgboost-split_1675457761144/work/src/learner.cc:888: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
Not enough frames for bud 311 at time 119. Only 1 frames available.
Not enough frames for bud 312 at time 119. Only 1 frames available.
Not enough frames for bud 313 at time 119. Only 1 frames available.
Not enough frames for bud 314 at time 119. Only 1 frames available.
Not enough frames for bud 315 at time 119. Only 1 frames available.
Not enough frames for bud 316 at time 119. Only 1 frames available.
Not enough frames for bud 317 at time 119. Only 1 frames available.
Not enough frames for bud 318 at time 119. Only 1 frames available.
Not enough frames for bud 319 at time 119. Only 1 frames available.
Not enough frames for bud 320 at time 119. Only 1 frames available.
Not enough frames for bud 321 at time 119. Only 1 frames available.
Not enough

In [17]:
external_subset_features['colony'] = 10
external_subset_lineage_gt = pd.read_csv(external_subset_lineage_GT_path_edited)
external_subset_lineage_gt['colony'] = 10
path_to_features = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/'

external_subset_features.to_csv(os.path.join(
    path_to_features, 'external_subset_candidate_features_{}_{}_nn_{}_features.csv'.format(threshold_mode, num_nn, 19)))

In [18]:
# create matrix features for the external subset
external_subset_matrix_features = get_custom_matrix_features(
    external_subset_features, external_subset_lineage_gt, f_list)

# train
# baseline (0.8486527929901424 +/- 0.01593595415915115)
f_list = ['dist_0', 'dist_max','poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_first','position_bud_std','orientation_bud_first','orientation_bud_last', 'plyfit_orientation_bud']
matrix_feat_custom = get_custom_matrix_features(all_candidate_features, all_gt, f_list)
config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all'}
models, accuracies = cv_nn(matrix_feat_custom, config=config)

# test the model on the external subset
accuracies = []
best_accuracy = 0
best_model = None
for model in models:
    pred, accuracy = test_nn(model, external_subset_matrix_features)
    accuracies.append(accuracy)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
print('accuracy on external subset: ', np.mean(accuracies))


config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_first', 'position_bud_std', 'orientation_bud_first', 'orientation_bud_last', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 1}
yes. I am reloaded


best_accuracy,▁▄▅▆▇███████████████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
eval_accuracy,▁▄▅▆▇██▆████▆▆▇▇▇▇▅▇
lr,██▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▁▁
patience,▁▁▁▁▁▁▂▂▁▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▆▆▆▇▇▇▇▇▇██▇▇█████
best_accuracy,0.8303
epoch,19
eval_accuracy,0.81212
lr,0.008
patience,10


early stopping at  28 LR:  0.0071029000000000005
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_first', 'position_bud_std', 'orientation_bud_first', 'orientation_bud_last', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 2}
yes. I am reloaded


best_accuracy,▁▄▅▅▅▇▇▇▇▇▇▇▇▇▇▇▇███████████
epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
eval_accuracy,▁▄▅▃▃▇▅▄▆▇▅▇▆▇▆▇▇█▇█▅▅▇▆▆▇▆▅
lr,██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
patience,▁▁▁▂▂▁▂▂▃▁▂▂▃▄▅▅▆▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█████████
best_accuracy,0.86145
epoch,27
eval_accuracy,0.81325
lr,0.0072
patience,10


early stopping at  21 LR:  0.0078022000000000005
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_first', 'position_bud_std', 'orientation_bud_first', 'orientation_bud_last', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 3}
yes. I am reloaded


...

best_accuracy,▁▃▆▆▆▆▆▆▇▇██████████████████
epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
eval_accuracy,▁▃▆▆▆▆▆▆▇▇█▇▇▇▆█▇█▇▇▇▅█▇▇█▇▇
lr,██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
patience,▁▁▁▂▁▂▂▁▁▂▁▂▂▃▄▁▂▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▅▆▆▇▇▇▇▇▇▇▇▇████████▇█████
best_accuracy,0.86061
epoch,27
eval_accuracy,0.83636
lr,0.0072
patience,10


early stopping at  25 LR:  0.0074026000000000005
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_max', 'poly_fit_budcm_candidcm', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_first', 'position_bud_std', 'orientation_bud_first', 'orientation_bud_last', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 5}
yes. I am reloaded


best_accuracy,▁▁▁▁▁▂▅▆▆▆▇▇▇▇███████████
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
eval_accuracy,▄▁▁▃▂▅▆▇▁▅██▇▆█▄▇██▇▃▇▅▇█
lr,██▇▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁
patience,▁▂▂▃▄▁▁▁▂▂▁▂▂▃▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇████████
best_accuracy,0.83636
epoch,24
eval_accuracy,0.8303
lr,0.0075
patience,10


early stopping at  15 LR:  0.0084016
accuracy:  0.8474406717780212 +/- 0.012545599873562093


RuntimeError: mat1 and mat2 shapes cannot be multiplied (295x64 and 40x64)

In [ ]:
best_model_path = '/home/farzaneh/Documents/Bread/bread/src/bread/algo/lineage/saved_models'
name = 'bst_for_external_16_features[64,64,5].pth'

In [ ]:
accuracies = []
for model in models:
    pred, accuracy = test_nn(model, matrix_feat_custom)
    accuracies.append(accuracy)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
print('accuracy on external subset: ', np.mean(accuracies))

# Run until we get accuracy more than 86. Then save all above 86 accuracies.

# Meanwhile, test on external and save models with more than 82.


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nn_model import cv_nn 

In [19]:
import torch
best_internal_accuracy = 0
best_external_accuracy = 0
save_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models'
best_model = None
f_list = ['dist_0','dist_std','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']
matrix_feat_custom = get_custom_matrix_features(all_candidate_features, all_gt, f_list,)
external_subset_matrix_features = get_custom_matrix_features(
    external_subset_features, external_subset_lineage_gt, f_list)
for seed in range(43):
    external_accuracies = []
    config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all','seed':seed}
    models, accuracies = cv_nn(matrix_feat_custom, config=config, seed=seed)
    if(np.mean(accuracies)>best_internal_accuracy):
        best_internal_accuracy = np.mean(accuracies)
        torch.save(model.state_dict(), os.path.join(save_path, 'best_model_on_internal_seed{}_{}_lr{}_b{}_.pth'.format(seed, config['layers'], config['lr'], config['batch_size'])))
        # test this on external subset
        for model in models:
            pred, accuracy = test_nn(model, external_subset_matrix_features)
            external_accuracies.append(accuracy)
            if(accuracy>best_external_accuracy):
            
                best_external_accuracy = accuracy
                best_model = model
                # save the model
                torch.save(model.state_dict(), os.path.join(save_path, 'best_model_on_external_seed{}_{}_lr{}_b{}_.pth'.format(seed, config['layers'], config['lr'], config['batch_size'])))
    if((best_internal_accuracy > 0.866) & (best_external_accuracy > 0.835)):
        break
                

config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'seed': 0, 'cv_number': 1}
yes. I am reloaded


best_accuracy,▁▂▇▇███████████
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
eval_accuracy,▁▂▇▄█▅▇▅▅▆▅█▅▇█
lr,██▇▇▆▅▅▅▄▄▃▃▂▂▁
patience,▁▁▁▂▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▆▆▆▇▇▇▇▇▇████
best_accuracy,0.8303
epoch,14
eval_accuracy,0.8303
lr,0.0085
patience,10


early stopping at  20 LR:  0.0079021
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'seed': 0, 'cv_number': 2}
yes. I am reloaded


best_accuracy,▁▇▇▇▇▇▇▇▇███████████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
eval_accuracy,▃▇▆▇▆▇▁▇▆█▅▆▅▅▆▆▆▅▆█
lr,██▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▁▁
patience,▁▁▂▂▃▄▅▅▆▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▆▆▆▆▆▇▇▇▇▇███████
best_accuracy,0.84337
epoch,19
eval_accuracy,0.84337
lr,0.008
patience,10


early stopping at  31 LR:  0.0068032000000000006
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'seed': 0, 'cv_number': 3}
yes. I am reloaded


...

early stopping at  37 LR:  0.006203800000000001
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'seed': 32, 'cv_number': 4}
yes. I am reloaded


best_accuracy,▁▂▃▃▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████████
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
eval_accuracy,▁▂▃▃▅▄▆▅▄▆▄▆▆▆▆▆▇▆▇▆▆▆▆▆▆▆█▆▆▆▆▆▆▆▆▇▆
lr,███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
patience,▁▁▁▂▁▂▁▂▂▃▄▁▂▂▃▄▁▂▁▂▂▃▄▅▅▆▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▅▅▆▆▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇█▇▇▇█████████
best_accuracy,0.90909
epoch,36
eval_accuracy,0.87273
lr,0.0063
patience,10


early stopping at  36 LR:  0.006303700000000001
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'seed': 32, 'cv_number': 5}
yes. I am reloaded


best_accuracy,▁▁▁▃▃▃▃▃▄▄▄▄▄▄▄▄▄▆▆▆▆▆▆▆▆███████████
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
eval_accuracy,▃▁▁▄▄▃▃▄▅▅▃▅▅▃▄▃▄▆▃▆▅▄▄▆▆█▅▆▇▅▅▃▅▆▆▆
lr,███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
patience,▁▂▂▁▂▂▃▄▁▂▂▃▄▅▅▆▇▁▂▂▃▄▅▅▆▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▅▅▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████
best_accuracy,0.85455
epoch,35
eval_accuracy,0.8303
lr,0.0064
patience,10


early stopping at  23 LR:  0.0076024000000000005
accuracy:  0.8680540343190944 +/- 0.021026124418088522
test accuracy 0.8169491525423729
test accuracy 0.7491525423728813
test accuracy 0.7627118644067796
test accuracy 0.8440677966101695
test accuracy 0.8305084745762712


# test with saved models


In [25]:
import pathlib
save_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models'
save_path = pathlib.Path(save_path)
model = LineageNN([64,64,5]) 

for file in save_path.iterdir():
    print(file)
    model.load_state_dict(torch.load(file))
    model.eval()
    pred, accuracy = test_nn(model, external_subset_matrix_features)
    print(file, accuracy)

/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models/best_model_on_external_seed0_[64, 64, 5]_lr0.01_b256_.pth
test accuracy 0.8440677966101695
/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models/best_model_on_external_seed0_[64, 64, 5]_lr0.01_b256_.pth 0.8440677966101695
/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models/best_model_on_internal_seed31_[64, 64, 5]_lr0.01_b256_.pth
test accuracy 0.8203389830508474
/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models/best_model_on_internal_seed31_[64, 64, 5]_lr0.01_b256_.pth 0.8203389830508474
/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models/best_model_on_internal_seed32_[64, 64, 5]_lr0.01_b256_.pth
test accuracy 0.7661016949152543
/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models/best_model_on_internal_seed32_[64, 64, 5]_lr0.01_b256_.pth 0.7661016949152543
/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models/best_model_on_internal_seed9_[

RuntimeError: Error(s) in loading state_dict for LineageNN:
	size mismatch for layers.0.weight: copying a param with shape torch.Size([64, 40]) from checkpoint, the shape in current model is torch.Size([64, 64]).

In [27]:
import pathlib
save_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models'
save_path = pathlib.Path(save_path)
model = LineageNN([64,64,5]) 
f_list = ['dist_0','dist_std','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']
matrix_feat_custom = get_custom_matrix_features(all_candidate_features, all_gt, f_list,)

for file in save_path.iterdir():
    print(file)
    model.load_state_dict(torch.load(file))
    model.eval()
    pred, accuracy = test_nn(model, matrix_feat_custom)
    print(file, accuracy)

/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models/best_model_on_external_seed0_[64, 64, 5]_lr0.01_b256_.pth
test accuracy 0.8498789346246973
/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models/best_model_on_external_seed0_[64, 64, 5]_lr0.01_b256_.pth 0.8498789346246973
/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models/best_model_on_internal_seed31_[64, 64, 5]_lr0.01_b256_.pth
test accuracy 0.8595641646489104
/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models/best_model_on_internal_seed31_[64, 64, 5]_lr0.01_b256_.pth 0.8595641646489104
/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models/best_model_on_internal_seed32_[64, 64, 5]_lr0.01_b256_.pth
test accuracy 0.8583535108958837
/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models/best_model_on_internal_seed32_[64, 64, 5]_lr0.01_b256_.pth 0.8583535108958837
/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models/best_model_on_internal_seed9_[

RuntimeError: Error(s) in loading state_dict for LineageNN:
	size mismatch for layers.0.weight: copying a param with shape torch.Size([64, 40]) from checkpoint, the shape in current model is torch.Size([64, 64]).

# run best model and see errors


In [30]:
import pathlib
save_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/saved_models'
save_path = pathlib.Path(save_path)
model = LineageNN([64,64,5]) 
f_list = ['dist_0','dist_std','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']
external_subset_matrix_features = get_custom_matrix_features(
    external_subset_features, external_subset_lineage_gt, f_list)
file = '/home/farzaneh/Documents/Bread/bread/src/bread/algo/lineage/saved_models/best_nn_all_4frames_16features_seed9_[64, 64, 5]_lr0.01_b256.pth'
model.load_state_dict(torch.load(file))
model.eval()
pred, accuracy = test_nn(model, external_subset_matrix_features)
errors = pred.loc[pred['predicted'] != pred['parent_index_in_candidates']]
errors.reset_index(drop=True, inplace=True)
errors


test accuracy 0.8135593220338984


,Unnamed: 0,parent_GT,bud_id,time_index,colony,features,candidates,parent_index_in_candidates,predicted
0,10,3,11,30,10,"[[1.0, 1.2509881770620215, 1.682411017497046, ...","[3, 6, 8, 1]",0,1
1,13,2,14,36,10,"[[1.0, 0.17935973380357526, 1.458820288786745,...","[2, 5, 9, 8]",0,3
2,17,3,18,46,10,"[[1.0, 1.1145180572670008, 1.514606954697833, ...","[3, 7, 11, 13]",0,1
3,19,1,20,48,10,"[[1.0, 0.7524491055482474, 1.4097651186084885,...","[1, 12, 17, 4]",0,1
4,25,5,26,52,10,"[[1.0, 0.8660254037844386, 0.8705906892628679,...","[5, 16, 14, 19]",0,1
5,26,7,27,54,10,"[[1.0, 0.5, 1.0548030138938482, 2.010023947312...","[7, 12, 15, 11]",0,3
6,30,14,31,59,10,"[[1.0, 0.4330127018922193, 1.5230728126607673,...","[2, 8, 14, 1]",2,0
7,34,3,35,61,10,"[[1.0, 1.52481577015833, 2.1083405084622058, 2...","[3, 7, 18, 13]",0,1
8,49,28,50,72,10,"[[1.0, 0.7917321694993698, 0.8141582732851511,...","[10, 28, 37, 4]",1,0
9,52,8,53,74,10,"[[1.0, 0.6180339887498949, 0.9201187942174467,...","[6, 8, 34, 5]",1,2


# Train and test on all_plus
